In [7]:
import random
import pandas as pd

NUM_BUSSES = 8
NUM_OPERATORS = 10
SHIFT_8HR = 5
SHIFT_12HR = NUM_OPERATORS - SHIFT_8HR
SHIFT_TIMES = {"8ч": 8, "12ч": 12}
WEEK_DAYS = ["Понедельник", "Вторник", "Среда", "Четверг", "Пятница", "Суббота", "Воскресенье"]
HOURS_OF_DAY = range(24)

POPULATION_SIZE = 50
MAX_GEN_COUNT = 100
MUTATION_PROB = 0.1
CROSSOVER_PROB = 0.8

def check_rest_day(operator_id, day_idx):
    if operator_id < SHIFT_8HR:
        return WEEK_DAYS[day_idx] in ["Суббота", "Воскресенье"]
    return day_idx % 3 != 0

def create_random_schedule():
    weekly_schedule = []
    for day_idx, day in enumerate(WEEK_DAYS):
        daily_shifts = []
        for operator_id in range(NUM_OPERATORS):
            if check_rest_day(operator_id, day_idx):
                daily_shifts.append({"operator": operator_id, "bus": None, "start": None, "end": None, "rest": True})
            else:
                bus = random.randint(0, NUM_BUSSES - 1)
                shift_type = "8ч" if operator_id < SHIFT_8HR else "12ч"
                start_hour = random.randint(6, 10) if shift_type == "8ч" else random.randint(0, 23)
                end_hour = (start_hour + SHIFT_TIMES[shift_type]) % 24
                daily_shifts.append({
                    "operator": operator_id,
                    "bus": bus,
                    "start": start_hour,
                    "end": end_hour,
                    "rest": False
                })
        weekly_schedule.append(daily_shifts)
    return weekly_schedule

def evaluate_schedule(weekly_schedule):
    penalty = 0
    bus_coverage = {day: {hour: [] for hour in HOURS_OF_DAY} for day in WEEK_DAYS}
    
    for day_idx, daily_shifts in enumerate(weekly_schedule):
        for shift in daily_shifts:
            if shift["rest"]:
                continue
            bus = shift["bus"]
            start = shift["start"]
            end = shift["end"]
            
            if bus is not None:
                hours = range(start, end + 1 if end >= start else end + 24)
                for hour in hours:
                    bus_coverage[WEEK_DAYS[day_idx]][hour % 24].append(bus)
            
            if shift["operator"] < SHIFT_8HR and WEEK_DAYS[day_idx] in ["Суббота", "Воскресенье"]:
                penalty += 10
            if shift["operator"] >= SHIFT_8HR:
                if day_idx > 0:
                    prev_day = weekly_schedule[day_idx - 1]
                    for prev_shift in prev_day:
                        if prev_shift["operator"] == shift["operator"] and not prev_shift["rest"]:
                            penalty += 10
    
    for day, coverage in bus_coverage.items():
        for hour, buses in coverage.items():
            if len(set(buses)) < NUM_BUSSES:
                penalty += 5 * (NUM_BUSSES - len(set(buses)))
    
    return -penalty

def perform_crossover(parent1, parent2):
    crossover_point = random.randint(0, len(parent1) - 1)
    child = parent1[:crossover_point] + parent2[crossover_point:]
    return child

def apply_mutation(weekly_schedule):
    day_idx = random.randint(0, len(weekly_schedule) - 1)
    operator_idx = random.randint(0, NUM_OPERATORS - 1)
    shift_type = "8ч" if operator_idx < SHIFT_8HR else "12ч"
    shift_duration = SHIFT_TIMES[shift_type]
    bus = random.randint(0, NUM_BUSSES - 1)
    start_hour = random.randint(6, 10) if shift_type == "8ч" else random.randint(0, 23)
    end_hour = (start_hour + shift_duration) % 24
    weekly_schedule[day_idx][operator_idx] = {
        "operator": operator_idx,
        "bus": bus,
        "start": start_hour,
        "end": end_hour,
        "rest": False
    }
    return weekly_schedule

def run_genetic_algorithm():
    population = [create_random_schedule() for _ in range(POPULATION_SIZE)]
    for generation in range(MAX_GEN_COUNT):
        population = sorted(population, key=evaluate_schedule, reverse=True)
        next_generation = population[:5]
        
        while len(next_generation) < POPULATION_SIZE:
            if random.random() < CROSSOVER_PROB:
                parent1, parent2 = random.sample(population[:20], 2)
                child = perform_crossover(parent1, parent2)
            else:
                child = random.choice(population[:20])
            if random.random() < MUTATION_PROB:
                child = apply_mutation(child)
            next_generation.append(child)
        
        population = next_generation
        print(f"Поколение {generation + 1}: Лучшая оценка = {evaluate_schedule(population[0])}")
    
    return population[0]

def create_schedule_table(weekly_schedule, num_8hr_drivers, num_12hr_drivers):
    days = ["Понедельник", "Вторник", "Среда", "Четверг", "Пятница", "Суббота", "Воскресенье"]
    operator_labels = [f"Водитель {d + 1}" for d in range(NUM_OPERATORS)]
    shifts = ["8ч" if d < num_8hr_drivers else "12ч" for d in range(NUM_OPERATORS)]

    schedule_df = pd.DataFrame(index=operator_labels, columns=["Смена"] + days)
    schedule_df["Смена"] = shifts

    for day_idx, daily_shifts in enumerate(weekly_schedule):
        for shift in daily_shifts:
            operator = f"Водитель {shift['operator'] + 1}"
            if shift["rest"]:
                schedule_df.loc[operator, days[day_idx]] = "Отдых"
            else:
                bus = shift["bus"]
                start = shift["start"]
                end = shift["end"]
                schedule_df.loc[operator, days[day_idx]] = f"Автобус {bus + 1} ({start:02d}:00 - {end:02d}:00)"

    schedule_df = schedule_df.sort_values(by="Смена", ascending=False)
    
    return schedule_df


def style_schedule_table(schedule_df):
    styled_schedule = (
        schedule_df.style.set_properties(
            **{
                "text-align": "center",
                "font-size": "14px",  
                "border": "1px solid #ddd",  
                "padding": "8px",  
                "background-color": "#f5f5f5",  
            }
        )
        .set_table_styles(
            [
                
                {"selector": "th", "props": [("background-color", "#4CAF50"), ("color", "white"), ("font-size", "16px"), ("font-weight", "bold")]},
                
                {"selector": "td", "props": [("background-color", "#ffffff"), ("font-size", "14px"), ("font-weight", "normal"), ("color", "#333333")]},
                {"selector": "tr:nth-child(odd)", "props": [("background-color", "#f9f9f9")]},
                {"selector": "tr:nth-child(even)", "props": [("background-color", "#ffffff")]},
                
                {"selector": "td, th", "props": [("border", "1px solid #ddd")]},
                
                {"selector": "th", "props": [("padding", "10px")]},
                {"selector": "td", "props": [("padding", "10px")]},
            ]
        )
    )
    
    return styled_schedule

optimal_schedule = run_genetic_algorithm()

schedule_df = create_schedule_table(optimal_schedule, num_8hr_drivers=SHIFT_8HR, num_12hr_drivers=SHIFT_12HR)
styled_schedule = style_schedule_table(schedule_df)
styled_schedule


Поколение 1: Лучшая оценка = -4805
Поколение 2: Лучшая оценка = -4835
Поколение 3: Лучшая оценка = -4800
Поколение 4: Лучшая оценка = -4705
Поколение 5: Лучшая оценка = -4745
Поколение 6: Лучшая оценка = -4725
Поколение 7: Лучшая оценка = -4610
Поколение 8: Лучшая оценка = -4515
Поколение 9: Лучшая оценка = -4455
Поколение 10: Лучшая оценка = -4480
Поколение 11: Лучшая оценка = -4470
Поколение 12: Лучшая оценка = -4405
Поколение 13: Лучшая оценка = -4525
Поколение 14: Лучшая оценка = -4485
Поколение 15: Лучшая оценка = -4530
Поколение 16: Лучшая оценка = -4565
Поколение 17: Лучшая оценка = -4495
Поколение 18: Лучшая оценка = -4480
Поколение 19: Лучшая оценка = -4460
Поколение 20: Лучшая оценка = -4490
Поколение 21: Лучшая оценка = -4395
Поколение 22: Лучшая оценка = -4350
Поколение 23: Лучшая оценка = -4300
Поколение 24: Лучшая оценка = -4405
Поколение 25: Лучшая оценка = -4495
Поколение 26: Лучшая оценка = -4505
Поколение 27: Лучшая оценка = -4445
Поколение 28: Лучшая оценка = -4440
П

,Смена,Понедельник,Вторник,Среда,Четверг,Пятница,Суббота,Воскресенье
Водитель 1,8ч,Автобус 8 (07:00 - 15:00),Автобус 6 (07:00 - 15:00),Автобус 2 (09:00 - 17:00),Автобус 6 (10:00 - 18:00),Автобус 2 (10:00 - 18:00),Автобус 4 (09:00 - 17:00),Автобус 8 (09:00 - 17:00)
Водитель 2,8ч,Автобус 5 (10:00 - 18:00),Автобус 1 (06:00 - 14:00),Автобус 2 (09:00 - 17:00),Автобус 5 (10:00 - 18:00),Автобус 3 (09:00 - 17:00),Автобус 2 (07:00 - 15:00),Автобус 2 (09:00 - 17:00)
Водитель 3,8ч,Автобус 6 (10:00 - 18:00),Автобус 8 (09:00 - 17:00),Автобус 6 (10:00 - 18:00),Автобус 6 (10:00 - 18:00),Автобус 8 (09:00 - 17:00),Автобус 6 (07:00 - 15:00),Автобус 4 (10:00 - 18:00)
Водитель 4,8ч,Автобус 4 (06:00 - 14:00),Автобус 6 (10:00 - 18:00),Автобус 8 (07:00 - 15:00),Автобус 3 (09:00 - 17:00),Автобус 1 (06:00 - 14:00),Автобус 2 (08:00 - 16:00),Автобус 4 (09:00 - 17:00)
Водитель 5,8ч,Автобус 3 (09:00 - 17:00),Автобус 2 (09:00 - 17:00),Автобус 3 (06:00 - 14:00),Автобус 8 (08:00 - 16:00),Автобус 7 (09:00 - 17:00),Автобус 5 (08:00 - 16:00),Автобус 4 (10:00 - 18:00)
Водитель 6,12ч,Автобус 2 (17:00 - 05:00),Автобус 4 (13:00 - 01:00),Автобус 5 (01:00 - 13:00),Автобус 5 (02:00 - 14:00),Автобус 3 (13:00 - 01:00),Автобус 1 (10:00 - 22:00),Автобус 4 (19:00 - 07:00)
Водитель 7,12ч,Автобус 8 (11:00 - 23:00),Автобус 7 (10:00 - 22:00),Автобус 5 (01:00 - 13:00),Автобус 8 (11:00 - 23:00),Автобус 5 (20:00 - 08:00),Автобус 8 (18:00 - 06:00),Автобус 2 (13:00 - 01:00)
Водитель 8,12ч,Автобус 1 (07:00 - 19:00),Автобус 1 (12:00 - 00:00),Автобус 7 (00:00 - 12:00),Автобус 1 (09:00 - 21:00),Автобус 3 (20:00 - 08:00),Автобус 8 (00:00 - 12:00),Автобус 6 (03:00 - 15:00)
Водитель 9,12ч,Автобус 8 (11:00 - 23:00),Автобус 4 (00:00 - 12:00),Автобус 8 (04:00 - 16:00),Автобус 2 (10:00 - 22:00),Автобус 6 (15:00 - 03:00),Автобус 1 (22:00 - 10:00),Автобус 3 (14:00 - 02:00)
Водитель 10,12ч,Автобус 3 (04:00 - 16:00),Автобус 4 (19:00 - 07:00),Автобус 6 (20:00 - 08:00),Автобус 1 (10:00 - 22:00),Автобус 5 (16:00 - 04:00),Автобус 6 (04:00 - 16:00),Автобус 8 (17:00 - 05:00)
